In [1]:
versioninfo()

Julia Version 1.5.0
Commit 96786e22cc (2020-08-01 23:44 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)


In [2]:
fizz(val::Int) = val % 3 == 0 ? "Fizz" : val
fizz(val) = val

fizz (generic function with 2 methods)

In [3]:
buzz(val::Int) = val % 5 == 0 ? "Buzz" : val
buzz(val) = val

buzz (generic function with 2 methods)

In [4]:
for n = 1:15
    n |> fizz |> println
end

1
2
Fizz
4
5
Fizz
7
8
Fizz
10
11
Fizz
13
14
Fizz


In [5]:
for n = 1:15
    n |> buzz |> println
end

1
2
3
4
Buzz
6
7
8
9
Buzz
11
12
13
14
Buzz


In [6]:
for n = 1:15
    n |> fizz |> buzz |> println
end

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
Fizz


↑最後の出力を `"FizzBuzz"` にしたい！

In [7]:
struct WrappedFizzBuzzFn0d{F}
    fn::F
    val::Int
end
(fn::WrappedFizzBuzzFn0d)(val::Int) = fn.fn(val)
(fn::WrappedFizzBuzzFn0d)(val) = _combine(val, fn.fn(fn.val))

_combine(n::Int, ::Int) = n
_combine(val::String, ::Int) = val
_combine(::Int, val::String) = val
_combine(val1::String, val2::String) = val1 * val2

_combine (generic function with 4 methods)

In [8]:
struct FizzBuzzStyle{N} <: Broadcast.AbstractArrayStyle{N} end

In [9]:
const _FizzBuzzFn = Union{typeof(fizz), typeof(buzz)}
const FizzBuzzFn = Union{_FizzBuzzFn, WrappedFizzBuzzFn0d{<:_FizzBuzzFn}}

Union{typeof(buzz), typeof(fizz), WrappedFizzBuzzFn0d{var"#s3"} where var"#s3"<:Union{typeof(buzz), typeof(fizz)}}

In [10]:
Broadcast.broadcasted(fn::FizzBuzzFn, n::Int) = 
    Broadcast.Broadcasted{FizzBuzzStyle{0}}(WrappedFizzBuzzFn0d(fn, n), (n,))
Broadcast.broadcasted(fn::FizzBuzzFn, bc::Broadcast.Broadcasted{FizzBuzzStyle{0}}) = 
    Broadcast.Broadcasted{FizzBuzzStyle{0}}(WrappedFizzBuzzFn0d(fn, bc.f.val), (bc,))

In [11]:
bc1 = Broadcast.broadcasted(fizz, 15)
bc2 = Broadcast.broadcasted(buzz, bc1)

Base.Broadcast.Broadcasted(WrappedFizzBuzzFn0d{typeof(buzz)}(buzz, 15), (Base.Broadcast.Broadcasted(WrappedFizzBuzzFn0d{typeof(fizz)}(fizz, 15), (15,)),))

In [12]:
bc1[]

"Fizz"

In [13]:
bc2[]

"FizzBuzz"

In [14]:
3 .|> fizz .|> buzz

"Fizz"

In [15]:
5 .|> fizz .|> buzz

"Buzz"

In [16]:
15 .|> fizz .|> buzz

"Fizz"

In [17]:
Broadcast.broadcasted(::typeof(|>), val, fn::FizzBuzzFn) = Broadcast.broadcasted(fn, val)

In [18]:
15 .|> fizz .|> buzz

"FizzBuzz"

In [19]:
for n = 1:15
    n .|> fizz .|> buzz .|> println
end

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz


In [20]:
1:15 .|> fizz .|> buzz .|> println;

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
Fizz


↑これも最後の出力を `"FizzBuzz"` にしたい！

In [21]:
struct WrappedFizzBuzzFnNd{N}
    fn::FizzBuzzFn
    val::AbstractArray{Int, N}
end
Base.getindex(fn::WrappedFizzBuzzFnNd, idx) = WrappedFizzBuzzFn0d(fn.fn, fn.val[idx])

In [22]:
Broadcast.broadcasted(fn::FizzBuzzFn, val::AbstractArray{Int, N}) where {N} = 
    Broadcast.Broadcasted{FizzBuzzStyle{N}}(WrappedFizzBuzzFnNd{N}(fn, val), (val,))
Broadcast.broadcasted(fn::FizzBuzzFn, bc::Broadcast.Broadcasted{FizzBuzzStyle{N}}) where {N} = 
    Broadcast.Broadcasted{FizzBuzzStyle{N}}(WrappedFizzBuzzFnNd{N}(fn, bc.f.val), (bc,))

In [23]:
Broadcast.broadcasted(fn::FizzBuzzFn, bc::Broadcast.Broadcasted{<:Broadcast.AbstractArrayStyle{N}}) where {N} = 
    Broadcast.Broadcasted{FizzBuzzStyle{N}}(WrappedFizzBuzzFnNd{N}(fn, copy(bc)), (bc,))

In [24]:
Broadcast.broadcasted(fn::FizzBuzzFn, v::Broadcast.Extruded{<:AbstractArray{T, N}}) where {T, N} = 
    Broadcast.Broadcasted{FizzBuzzStyle{N}}(WrappedFizzBuzzFnNd{N}(fn, v.x), (v.x,))

In [25]:
Base.similar(bc::Broadcast.Broadcasted{FizzBuzzStyle{N}}, ::Type{ElType}) where {N,ElType} =
    similar(Array{ElType}, axes(bc))

In [26]:
Base.@propagate_inbounds function Broadcast._broadcast_getindex(bc::Broadcast.Broadcasted{FizzBuzzStyle{0},<:Any,<:Any,<:Any}, I)
    args = Broadcast._getindex(bc.args, I)
    return Broadcast._broadcast_getindex_evalf(bc.f, args...)
end

In [27]:
Base.@propagate_inbounds function Broadcast._broadcast_getindex(bc::Broadcast.Broadcasted{FizzBuzzStyle{N},<:Any,<:Any,<:Any}, I) where {N}
    args = Broadcast._getindex(bc.args, I)
    return Broadcast._broadcast_getindex_evalf(bc.f[I], args...)
end

In [28]:
1:15 .|> fizz .|> buzz

15-element Array{Any,1}:
  1
  2
   "Fizz"
  4
   "Buzz"
   "Fizz"
  7
  8
   "Fizz"
   "Buzz"
 11
   "Fizz"
 13
 14
   "FizzBuzz"

In [29]:
1:15 .|> fizz .|> buzz .|> println

LoadError: MethodError: no method matching FizzBuzzStyle{1}(::Val{1})
Closest candidates are:
  FizzBuzzStyle{1}() where N at In[8]:1

In [30]:
(::Type{FizzBuzzStyle{N}})(::Val{M}) where {N, M} = Broadcast.DefaultArrayStyle{M}()

In [31]:
1:15 .|> fizz .|> buzz .|> println;

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz
